<a href="https://colab.research.google.com/github/SaideLeon/AlyBlog/blob/main/Qwen3_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# Qwen3-TTS — Clonagem de Voz com Interface em Português
#
# Modelo: Qwen3-TTS-12Hz-1.7B-Base
# Função: Voice Clone (Clonagem de Voz)
#
# Autor do tutorial base: 12.5 Tech
# Adaptação PT + Voice Clone: ChatGPT
# ============================================
# Execute cada célula sequencialmente

# ============================================
# CÉLULA 1: Instalação das dependências
# ============================================
!pip install -q qwen-tts flash-attn --no-build-isolation gradio soundfile

# ============================================
# CÉLULA 2: Importação das bibliotecas
# ============================================
import torch
import soundfile as sf
import gradio as gr
from qwen_tts import Qwen3TTSModel

# ============================================
# CÉLULA 3: Carregamento do modelo de clonagem
# ============================================
print("Carregando modelo de clonagem de voz...")
model = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-12Hz-1.7B-Base",
    device_map="cuda:0",
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)
print("Modelo carregado com sucesso!")

# ============================================
# CÉLULA 4: Função de clonagem de voz
# ============================================
def clonar_voz(
    novo_texto,
    idioma,
    audio_referencia,
    texto_referencia
):
    """
    Clona a voz a partir de um áudio de referência
    """

    if not novo_texto.strip():
        # Returning None for audio output when there's an error in text input
        return None, "❌ Insira o texto que será falado"

    if audio_referencia is None:
        # Returning None for audio output when there's an error in reference audio
        return None, "❌ Envie um áudio de referência"

    if not texto_referencia.strip():
        # Returning None for audio output when there's an error in reference text
        return None, "❌ Informe o texto falado no áudio de referência"

    try:
        # Geração da voz clonada
        wavs, sr = model.generate_voice_clone(
            text=novo_texto,
            language=idioma if idioma != "Auto" else "Auto",
            ref_audio=audio_referencia,
            ref_text=texto_referencia,
        )

        # Return audio data directly to Gradio
        return (wavs[0], sr), "✅ Voz clonada com sucesso!"

    except Exception as e:
        # Returning None for audio output when an exception occurs
        return None, f"Erro: {str(e)}"

# ============================================
# CÉLULA 5: Interface Gradio (100% em português)
# ============================================
with gr.Blocks(title="Clonagem de Voz — Qwen3-TTS") as demo:
    gr.Markdown("# 🎙️ Clonagem de Voz com IA (Qwen3-TTS)")
    gr.Markdown(
        "Clone uma voz real a partir de um áudio curto e gere novas falas com alta fidelidade."
    )
    gr.Markdown(" salió")

    with gr.Row():
        with gr.Column():
            # Texto que será falado pela voz clonada
            novo_texto = gr.Textbox(
                label="📝 Texto a ser falado",
                placeholder="Digite aqui o novo texto que a voz clonada irá falar...",
                lines=4
            )

            # Idioma
            idioma = gr.Dropdown(
                choices=[
                    "Auto", "Chinese", "English", "Japanese", "Korean",
                    "German", "French", "Russian", "Portuguese", "Spanish", "Italian"
                ],
                value="Auto",
                label="🌍 Idioma"
            )

            gr.Markdown("### 🎧 Áudio de referência (voz original)")

            # Upload do áudio de referência
            audio_referencia = gr.Audio(
                label="Áudio de referência",
                type="filepath"
            )

            # Texto falado no áudio de referência
            texto_referencia = gr.Textbox(
                label="📄 Texto falado no áudio",
                placeholder="Digite exatamente o que é falado no áudio enviado",
                lines=3
            )

            # Botão
            gerar_btn = gr.Button(
                "🎤 Clonar voz e gerar áudio",
                variant="primary",
                size="lg"
            )

        with gr.Column():
            # Change type to numpy to accept (audio_array, sample_rate) tuple
            audio_saida = gr.Audio(
                label="🔊 Áudio gerado",
                type="numpy"
            )
            status = gr.Textbox(
                label="Status",
                lines=2
            )

            # Ação do botão
            gerar_btn.click(
                fn=clonar_voz,
                inputs=[novo_texto, idioma, audio_referencia, texto_referencia],
                outputs=[audio_saida, status]
            )

# ============================================
# Inicialização da aplicação
# ============================================
demo.launch(share=True, debug=True)

In [ ]:
# Install SoX to handle audio processing dependencies
!apt-get update && apt-get install -y sox